In [1]:
import cpa
import scanpy as sc
import pandas as pd
import anndata as ad
import numpy as np

[rank: 0] Global seed set to 0
2024-10-03 09:17:55,828	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-10-03 09:17:55,986	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
split = 3

In [3]:
adata_train_path = f"/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/adata_train_{split}.h5ad"
adata_test_path = f"/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/adata_test_{split}.h5ad"
adata_ood_path = f"/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/adata_ood_{split}.h5ad"

In [4]:
adata_train = sc.read(adata_train_path)
adata_test = sc.read(adata_test_path)
adata_ood = sc.read(adata_ood_path)

In [5]:
adata_ood.obs["condition"].value_counts()

condition
Panobinostat+Curcumin      2244
Panobinostat+Sorafenib     2013
control+Dacinostat         1869
Panobinostat+SRT1720       1826
Panobinostat+Crizotinib    1641
control+Alvespimycin        758
SRT2104+Alvespimycin        520
control                     500
Name: count, dtype: int64

In [6]:
try:
    data_path = 'combo_sciplex_prep_hvg_filtered.h5ad'
    adata = sc.read(data_path)
except:
    import gdown
    gdown.download('https://drive.google.com/uc?export=download&id=1RRV0_qYKGTvD3oCklKfoZQFYqKJy4l6t')
    data_path = 'combo_sciplex_prep_hvg_filtered.h5ad'
    adata = sc.read(data_path)


adata.obs["index"] = adata.obs_names.values
adata.obs["index_adapted"] = adata.obs.apply(lambda x: x["index"][:-2], axis=1)
adata.obs.set_index("index_adapted", inplace=True)

adata_ood.obs["cov_drug_dose"] = adata.obs["cov_drug_dose"]
adata_ood.obs["condition_ID"] = adata.obs["condition_ID"]
adata_ood.obs["log_dose"] = adata.obs["log_dose"]
adata_ood.obs["smiles_rdkit"] = adata.obs["smiles_rdkit"]
adata_ood.obs["cell_type"] = adata.obs["cell_type"]

adata_test.obs["cov_drug_dose"] = adata.obs["cov_drug_dose"]
adata_test.obs["condition_ID"] = adata.obs["condition_ID"]
adata_test.obs["log_dose"] = adata.obs["log_dose"]
adata_test.obs["smiles_rdkit"] = adata.obs["smiles_rdkit"]
adata_test.obs["cell_type"] = adata.obs["cell_type"]

adata_train.obs["cov_drug_dose"] = adata.obs["cov_drug_dose"]
adata_train.obs["condition_ID"] = adata.obs["condition_ID"]
adata_train.obs["log_dose"] = adata.obs["log_dose"]
adata_train.obs["smiles_rdkit"] = adata.obs["smiles_rdkit"]
adata_train.obs["cell_type"] = adata.obs["cell_type"]

In [7]:
control_cells_gex = adata_ood[adata_ood.obs["condition"]=="control"].X.A
obs_names = list(adata_ood[adata_ood.obs["condition"]=="control"].obs_names)

all_data = []
obs_df = pd.DataFrame(columns=["condition_ID", "log_dose", "smiles_rdkit", "cell_type", "condition"])
indices = []

for condition in adata_ood.obs["condition"].cat.categories:
    if condition=="control":
        continue
    all_data.append(control_cells_gex)
    filtered_row = pd.Series(adata_ood[adata_ood.obs["condition"] == condition].obs[["condition_ID", "log_dose", "smiles_rdkit", "cell_type", "condition"]].iloc[0].values)
    filtered_row_df = pd.DataFrame([filtered_row.values], columns=["condition_ID", "log_dose", "smiles_rdkit", "cell_type", "condition"])
    duplicated_rows = pd.concat([filtered_row_df] * 500, ignore_index=True)
    obs_df = pd.concat([obs_df, duplicated_rows], ignore_index=True)

# Stack all data vertically to create a single array
all_data_array = np.vstack(all_data)


# Create the Anndata object
adata_ood_new = ad.AnnData(X=all_data_array, obs=obs_df)
adata_ood_new.var = adata_ood.var


control_cells_gex = adata_test[adata_test.obs["condition"]=="control"].X.A
obs_names = list(adata_test[adata_test.obs["condition"]=="control"].obs_names)

all_data = []
obs_df = pd.DataFrame(columns=["condition_ID", "log_dose", "smiles_rdkit", "cell_type", "condition"])
indices = []

for condition in adata_test.obs["condition"].cat.categories:
    if condition=="control":
        continue
    all_data.append(control_cells_gex)
    filtered_row = pd.Series(adata_test[adata_test.obs["condition"] == condition].obs[["condition_ID", "log_dose", "smiles_rdkit", "cell_type", "condition"]].iloc[0].values)
    filtered_row_df = pd.DataFrame([filtered_row.values], columns=["condition_ID", "log_dose", "smiles_rdkit", "cell_type", "condition"])
    duplicated_rows = pd.concat([filtered_row_df] * 500, ignore_index=True)
    obs_df = pd.concat([obs_df, duplicated_rows], ignore_index=True)

# Stack all data vertically to create a single array
all_data_array = np.vstack(all_data)


# Create the Anndata object
adata_test_new = ad.AnnData(X=all_data_array, obs=obs_df)
adata_test_new.var = adata_test.var

In [8]:
import anndata

adata_combined = anndata.concat((adata_train, adata_test_new, adata_ood_new), label="split", keys=["train", "test", "ood"], join="inner")

In [9]:
frac_valid = adata[adata.obs["split_1ct_MEC"] == "valid"].n_obs / (
    adata[adata.obs["split_1ct_MEC"] == "valid"].n_obs + adata[adata.obs["split_1ct_MEC"] == "train"].n_obs
)

In [10]:
import numpy as np


def create_split(x):
    if x["split"] != "train":
        return x["split"]
    is_train = np.random.choice(2, p=[frac_valid, 1 - frac_valid])
    if is_train:
        return "train_train"
    return "train_valid"


adata_combined.obs["split_1ct_MEC"] = adata_combined.obs.apply(create_split, axis=1)

In [11]:
cpa.CPA.setup_anndata(adata_combined,
                      perturbation_key='condition_ID',
                      dosage_key='log_dose',
                      control_group='CHEMBL504',
                      batch_key=None,
                      smiles_key='smiles_rdkit',
                      is_count_data=False,
                      categorical_covariate_keys=['cell_type'],
                      deg_uns_cat_key='cov_drug_dose',
                      max_comb_len=2,
                     )

100%|██████████| 65107/65107 [00:00<00:00, 982986.45it/s]
An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


INFO     Generating sequential column names                                                                        
INFO     Generating sequential column names                                                                        


In [12]:
df_hsearch = pd.read_csv("result_hsearch.csv").T
df_hsearch.columns = df_hsearch.iloc[0]
df_hsearch = df_hsearch.drop(df_hsearch.index[0])
config = df_hsearch.sort_values("r_squared", ascending=False).iloc[0]
del config["r_squared"]
config = config.to_dict()

In [13]:
config_adapted = {}
for k,v in config.items():
    if k=="adv_loss":
        config_adapted[k] = v
    elif isinstance(v, float): 
        if np.isnan(v):
            config_adapted[k] = None
        else:
            raise ValueError
    elif v == "True":
        config_adapted[k] = True
    elif v == "False":
        config_adapted[k] = False
    else:
        if float(v) == int(float(v)):
            config_adapted[k] = int(float(v))
        else:
            config_adapted[k] = float(v)

In [14]:
trainer_params = config_adapted

In [15]:
trainer_params

{'n_epochs_kl_warmup': None,
 'n_epochs_pretrain_ae': 30,
 'n_epochs_adv_warmup': 50,
 'n_epochs_mixup_warmup': 3,
 'mixup_alpha': 0.1,
 'adv_steps': None,
 'n_hidden_adv': 128,
 'n_layers_adv': 3,
 'use_batch_norm_adv': False,
 'use_layer_norm_adv': False,
 'dropout_rate_adv': 0.2,
 'reg_adv': 10,
 'pen_adv': 0.1,
 'lr': 0.0003,
 'wd': 4e-07,
 'adv_lr': 0.0003,
 'adv_wd': 4e-07,
 'adv_loss': 'cce',
 'doser_lr': 0.0003,
 'doser_wd': 4e-07,
 'do_clip_grad': False,
 'gradient_clip_value': 1,
 'step_size_lr': 10}

In [16]:
ae_hparams = {'n_latent': 64,
 'recon_loss': 'gauss',
 'doser_type': 'linear',
 'n_hidden_encoder': 256,
 'n_layers_encoder': 3,
 'n_hidden_decoder': 512,
 'n_layers_decoder': 2,
 'use_batch_norm_encoder': True,
 'use_layer_norm_encoder': False,
 'use_batch_norm_decoder': True,
 'use_layer_norm_decoder': False,
 'dropout_rate_encoder': 0.25,
 'dropout_rate_decoder': 0.25,
 'variational': False,
 'seed': 6478}


In [17]:
model = cpa.CPA(adata=adata_combined,
                split_key='split_1ct_MEC',
                train_split='train_train',
                valid_split='train_valid',
                test_split='ood',
                use_rdkit_embeddings=True,
                **ae_hparams,
               )

[rank: 0] Global seed set to 6478


(18, 2048)


In [18]:
model.train(max_epochs=200,
            use_gpu=True,
            batch_size=512,
            plan_kwargs=trainer_params,
            early_stopping_patience=10,
            check_val_every_n_epoch=5,
            save_path="/lustre/groups/ml01/workspace/ot_perturbation/models/chemcpa/combosciplex/1",
           )

100%|██████████| 32/32 [00:00<00:00, 133.53it/s]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 5/200:   2%|▏         | 4/200 [00:23<20:45,  6.35s/it, v_num=1, recon=-2.4e+3, r2_mean=0.917, adv_loss=2.42, acc_pert=0.156] 


Epoch 00004: cpa_metric reached. Module best state updated.


Epoch 10/200:   4%|▍         | 9/200 [01:07<26:12,  8.23s/it, v_num=1, recon=-2.87e+3, r2_mean=0.924, adv_loss=2.4, acc_pert=0.162, val_recon=-2.65e+3, disnt_basal=0.118, disnt_after=0.317, val_r2_mean=0.92, val_KL=nan]
disnt_basal = 0.10924570372217549
disnt_after = 0.2551900410611255
val_r2_mean = 0.921461926460266
val_r2_var = 0.7373096787929534
Epoch 20/200:  10%|▉         | 19/200 [02:37<26:33,  8.80s/it, v_num=1, recon=-3.13e+3, r2_mean=0.931, adv_loss=2.4, acc_pert=0.177, val_recon=-3e+3, disnt_basal=0.111, disnt_after=0.215, val_r2_mean=0.928, val_KL=nan]    
disnt_basal = 0.10297575350181057
disnt_after = 0.18506845604946456
val_r2_mean = 0.9296243261595567
val_r2_var = 0.7705651859641075
Epoch 30/200:  14%|█▍        | 29/200 [04:07<25:15,  8.86s/it, v_num=1, recon=-3.3e+3, r2_mean=0.933, adv_loss=2.41, acc_pert=0.174, val_recon=-3.19e+3, disnt_basal=0.0935, disnt_after=0.173, val_r2_mean=0.931, val_KL=nan] 
disnt_basal = 0.08694101221922597
disnt_after = 0.16388084418514498
v

In [19]:
model.predict(adata_combined)

100%|██████████| 2035/2035 [00:53<00:00, 38.04it/s] 


In [20]:
adata_combined.write(
    f"/lustre/groups/ml01/workspace/ot_perturbation/models/chemcpa/combosciplex/adata_with_predictions_{split}.h5ad"
)